In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader

In [9]:
import os
from langchain_community.document_loaders import UnstructuredPDFLoader

pdf_directory = "data"

file_paths = [os.path.join(pdf_directory, f)
              for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

documents = []
for file_path in file_paths:
    loader = UnstructuredPDFLoader(file_path)  # Create loader for each PDF
    pdf_docs = loader.load()  # Load the documents
    for doc in pdf_docs:
        # Add metadata with the file path as the source
        doc.metadata = {"source": file_path}
        documents.append(doc)  # Append to the final documents list

/Users/viruss/Project_ai_LLM/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [8]:
# Example: Print content and source of each document
for doc in documents:
    print(f"Source: {doc.metadata['source']}")

Source: data/NeuralNetworksAndDeepLearning.pdf
Source: data/0310_Python_Data_Science_Handbook.pdf
Source: data/2909_SQL-for-Data-Science.pdf
Source: data/Neural Networks 3rd Ed 2009.pdf
Source: data/1011_Practical-Linear-Algebra-for-Data-Science.pdf
Source: data/1110_AI_and_Machine_Learning_for_Coders_A_Programmers_Guide_to_Artificial.pdf
Source: data/0710_Scaling-Python-with-dask.pdf
Source: data/0608_python-for-data-science-for-dummies-3-ed.pdf


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200, chunk_overlap=300)
chunks = text_splitter.split_documents(documents)

print(f"Done splitting. Total chunks created: {len(chunks)}")

NameError: name 'documents' is not defined

In [12]:
import ollama
ollama.pull('nomic-embed-text')

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [5]:
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="Blog_LLM",
    persist_directory="vector_db"
)
print('done adding to vector database....')

NameError: name 'Chroma' is not defined

In [3]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI

from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
# llm = ChatGroq(model="llama-3.1-70b-versatile", streaming=True, groq_api_key="", max_tokens=6000)

In [4]:
streaming = True  # Define the streaming variable
llm = ChatOpenAI(model="gpt-4o-mini", streaming=streaming)

In [6]:
QUERY_PROMPT = PromptTemplate(
    input_variables=['question'],
    template="""
    You are an AI language model assistant. Your task is to generate five diffrent versions of the given user question to retrive relevant documents from
    a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitation of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}
    """
)

In [7]:
retriverer = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(),llm, prompt=QUERY_PROMPT
)

NameError: name 'vector_db' is not defined

In [96]:

template="""
    Answer the question based ONLY on the following context. Do not use any external information.
    If the question contains only non-relevant words such as articles (e.g., 'is', 'and', 'the', 'what', etc.), provide no answer.
    If the question contains significant keywords, generate a brief explanation of the relevant concept and provide the formula (if applicable).

    Context:
    {context}

    Question:
    {question}

    Explanation (brief) and Formula (if applicable):
    [Provide a concise explanation of the concept based on the context, and a formula or equation if applicable. If no formula is available, dont show the formula.]"]
    """

In [97]:
prompt = ChatPromptTemplate.from_template(template)

In [98]:
chain = (
    {"context": retriverer, "question": RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)

In [2]:
input_data = "backpropagation"

In [1]:
res = chain.invoke(input_data)
print(res)


NameError: name 'chain' is not defined